In [1]:
import pandas as pd 
import numpy as np
import datetime

# Travaux Perturbants la Circulation

Sur le site opendata.paris nous avons trouvé un dataset qui recense les travaux perturbants la circulation.

Nous récupérons les données qui concernent les axes que nous étudions pour connaitre les périodes et le niveau de perturbation de la voie.

In [2]:
# https://opendata.paris.fr/explore/dataset/chantiers-perturbants/information/?disjunctive.cp_arrondissement&disjunctive.maitre_ouvrage&disjunctive.objet&disjunctive.impact_circulation&disjunctive.niveau_perturbation&disjunctive.statut

travaux_circulation=pd.read_csv("/Users/daniel/Downloads/chantiers-perturbants.csv",sep=';')
travaux_circulation=travaux_circulation.rename(columns={"Voie(s)":"Voie"})
interest=[]

# sélectionner les axes qui nous intéressent
for i in travaux_circulation["Voie"].unique():
    if(type(i) is str):
        if("Champs Elysées" in i):
            interest.append(i) 
        if("Convention" in i):
            interest.append(i) 
        if("Sts-Pères" in i):
            interest.append(i) 

travaux_circulation=travaux_circulation[pd.DataFrame(travaux_circulation.Voie.tolist()).isin(interest).any(1).values]

# drop des colonnes inutiles
df=travaux_circulation[["Voie","Précisions de localisation","Date de début","Date de fin","Impact sur la circulation","Détail de l'impact sur la circulation","Niveau de perturbation","Statut"]]
df["Niveau de perturbation"]=df["Niveau de perturbation"].replace({2.0:"Perturbant",1.0:"Très perturbant"})
df["Statut"]=df["Statut"].replace({1:"A venir",2:"En cours",3:"Suspendu",4:"Prolongé",5:"Terminé"})
# export csv
df.to_csv("chantiers_perturbants_clean.csv")
df.head()

/var/folders/jx/28l9xggd2379trg_rfjvhlx80000gn/T/ipykernel_63742/2414427391.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Niveau de perturbation"]=df["Niveau de perturbation"].replace({2.0:"Perturbant",1.0:"Très perturbant"})
/var/folders/jx/28l9xggd2379trg_rfjvhlx80000gn/T/ipykernel_63742/2414427391.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Statut"]=df["Statut"].replace({1:"A venir",2:"En cours",3:"Suspendu",4:"Prolongé",5:"Terminé"})


,Voie,Précisions de localisation,Date de début,Date de fin,Impact sur la circulation,Détail de l'impact sur la circulation,Niveau de perturbation,Statut
33,Rue de la Convention,Au N°181,2022-07-04,2023-05-12,RESTREINTE,"Maintien de 3,20m dans chaque sens.",Très perturbant,En cours
41,Avenue des Champs Elysées,De la place de l'étoile à la place Clémenceau ...,2022-11-03,2023-02-24,RESTREINTE,Suppression d'une file de circulation de chaqu...,Perturbant,En cours
44,40 rue des Sts-Pères,juste avant le bd St Germain,2021-07-12,2022-12-31,RESTREINTE,Maintien d'une file de circulation.,Perturbant,En cours
79,Avenue des Champs Elysées,Au n°23 sur une partie du trottoir,2022-06-13,2023-12-01,RESTREINTE,Circulation piétonnes impactée avec entrées - ...,Perturbant,En cours


# Paris Respire

Le premier dimanche de chaque mois, l'avenue des champs elysées et piétonnisée

Paris respire prend effet entre 11h et 18h entre avril et octobre et entre 10h et 17h entre novembre et mars

In [3]:
import datetime
import numpy as np
import pandas as pd

paris_respire=pd.DataFrame(columns=["date","paris_repsire"])

list_first_sunday=[]
#premiers dimanches de chaque mois
for year in [2021,2022]:
        for month in ["%.2d" % i for i in range(1,13)]:
            list_first_sunday.append(np.busday_offset("{y}-{m}".format(y=year,m=month), 0, roll='forward', weekmask='Sun'))

dates = pd.DatetimeIndex(list_first_sunday)

#horaires (10-17 ou 11-18)
for s in dates:     
    if s.month <=8 and s.month >=4:
        for h in range(11,18):
            l=[s.replace(hour=h),True]
            paris_respire.loc[len(paris_respire)] = l
    else:
        for h in range(10,17):
            l=[s.replace(hour=h),True]
            paris_respire.loc[len(paris_respire)] = l

paris_respire.to_csv("paris_respire.csv")

paris_respire.head()


,date,paris_repsire
0,2021-01-03 10:00:00,True
1,2021-01-03 11:00:00,True
2,2021-01-03 12:00:00,True
3,2021-01-03 13:00:00,True
4,2021-01-03 14:00:00,True


# Pics de pollution

Il n'y a pas eu de modifications de réglementation concernant les vignettes critair au cours de la période que nous étudions.